# Hacer test de tu modelo

In [32]:
# Instalar tflite
!pip3 install --extra-index-url https://google-coral.github.io/py-repo/ tflite_runtime

Looking in indexes: https://pypi.org/simple, https://google-coral.github.io/py-repo/


In [33]:
# Cargar librerias necesarias
import tensorflow as tf
import tflite_runtime.interpreter as tflite
from PIL import Image, ImageOps
import numpy as np
import time

In [34]:
# Funcion para cargar el archivo de labels
def load_labels(filename):
  with open(filename, 'r') as f:
    return [line.strip() for line in f.readlines()]



In [62]:
# Simplifica la lectura de los datos suprimiendo la notacion cientifica
np.set_printoptions(suppress=True)


# Cargamos el modelo usando tflite
# model = tensorflow.keras.models.load_model('keras_model.h5')
interpreter = tf.lite.Interpreter(model_path="pokedex.tflite")
interpreter.allocate_tensors()

input_mean=127.5
input_std=127.5

# Obtenemos la referencia a los tensores de entrada y salida
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()


# Se verifica el tipo de tensor
floating_model = input_details[0]['dtype'] == np.float32

# Se define la "forma" de la entrada
input_shape = input_details[0]['shape']


# NxHxWxC, H:1, W:2
height = input_details[0]['shape'][1]
width = input_details[0]['shape'][2]

# Se ejecuta el modelo con una imagen de prueba
# Reemplaza esta imagen por una en formato JPG
# Leer imagenes png no funcionara con este ejemplo
#img = Image.open('charmander.jpg').resize((width, height))
#img = Image.open('charmeleon.jpg').resize((width, height))
#img = Image.open('charizard.jpg').resize((width, height))
#img = Image.open('bulbasaur.jpg').resize((width, height))
#img = Image.open('ivysaur.jpg').resize((width, height))
#img = Image.open('veusaur.jpg').resize((width, height))
#img = Image.open('squirtle.jpg').resize((width, height))
#img = Image.open('wartortle.jpg').resize((width, height))
#img = Image.open('blastoise.jpg').resize((width, height))


# add N dim
# https://numpy.org/doc/stable/reference/generated/numpy.expand_dims.html
# x  = np.array([1, 2])
# x.shape
# (2,)
# y = np.expand_dims(x, axis=0)
# y
# array([[1, 2]])
# y.shape
# (1, 2)
input_data = np.expand_dims(img, axis=0)

if floating_model:
    input_data = (np.float32(input_data) - input_mean) / input_std

# Se configura el tensor de entrada
interpreter.set_tensor(input_details[0]['index'], input_data)

# Se verfica el tiempo necesario para ejecutar una predicción
start_time = time.time()
# Se ejecuta el modelo
interpreter.invoke()
stop_time = time.time()

# Se obtiene el tensor de salida "prediccion"
output_data = interpreter.get_tensor(output_details[0]['index'])
# https://numpy.org/doc/stable/reference/generated/numpy.squeeze.html#numpy.squeeze
results = np.squeeze(output_data)

top_k = results.argsort()[-5:][::-1]
labels = load_labels("pokedex.txt")
for i in top_k:
    if floating_model:
        print('{:08.6f}: {}'.format(float(results[i]), labels[i]))
    else:
        print('{:08.6f}: {}'.format(float(results[i] / 255.0), labels[i]))

print('time: {:.3f}ms'.format((stop_time - start_time) * 1000))


1.000000: Charizard
0.000000: Charmeleon
0.000000: Charmader
0.000000: Venusaur
0.000000: Blastoise
time: 16.850ms
